# Lesson 6: Planning and Stock Report Generation

## Setup

In [1]:
from utils import get_openai_api_key
import os
OPENAI_API_KEY = get_openai_api_key()
llm_config = {"model": os.getenv('CHAT_MODEL'), "base_url": os.getenv('OPENAI_API_BASE')}

## The task!

In [2]:
task = "撰写一篇关于英伟达在过去一个月股票价格表现的博客文章。今天的日期是2024-08-02。"

## Build a group chat

This group chat will include these agents:

1. **User_proxy** or **Admin**: to allow the user to comment on the report and ask the writer to refine it.
2. **Planner**: to determine relevant information needed to complete the task.
3. **Engineer**: to write code using the defined plan by the planner.
4. **Executor**: to execute the code written by the engineer.
5. **Writer**: to write the report.

In [3]:
import autogen

In [4]:
user_proxy = autogen.ConversableAgent(
    name="管理员",
    system_message="给出任务，并发送 "
    "指示给作者以完善博客文章。",
    code_execution_config=False,
    llm_config=llm_config,
    human_input_mode="ALWAYS",
)

In [5]:
planner = autogen.ConversableAgent(
    name="规划者",
    system_message="给定一个任务，请确定完成该任务所需的信息。"
    "请注意，所有信息将通过 Python 代码检索。"
    "请仅建议可以通过 Python 代码检索的信息。"
    "在其他人完成每个步骤后，检查进度并指示剩余步骤。如果某个步骤失败，请尝试解决方案。",
    description="规划者。给定一个任务，确定完成该任务所需的信息。"
    "在其他人完成每个步骤后，检查进度并指示剩余步骤。",
    llm_config=llm_config,
)

In [6]:
engineer = autogen.AssistantAgent(
    name="工程师",
    llm_config=llm_config,
    description="一位根据规划者提供的计划编写代码的工程师。",
)

**Note**: In this lesson, you'll use an alternative method of code execution by providing a dict config. However, you can always use the LocalCommandLineCodeExecutor if you prefer. For more details about code_execution_config, check this: https://microsoft.github.io/autogen/docs/reference/agentchat/conversable_agent/#__init__

In [7]:
executor = autogen.ConversableAgent(
    name="执行者",
    system_message="执行工程师编写的代码并报告结果。",
    human_input_mode="NEVER",
    code_execution_config={
        "last_n_messages": 3,
        "work_dir": "coding",
        "use_docker": False,
    },
)

In [8]:
writer = autogen.ConversableAgent(
    name="写作者",
    llm_config=llm_config,
    system_message="写作者。"
    "请以 markdown 格式撰写博客（包含相关标题）"
    "并将内容放入伪 ```md``` 代码块中。"
    "您将接受管理员的反馈并完善您的博客。",
    description="写作者。"
    "根据代码执行结果撰写博客，并接受"
    "管理员的反馈以完善博客。"
)

## Define the group chat

In [9]:
groupchat = autogen.GroupChat(
    agents=[user_proxy, engineer, writer, executor, planner],
    messages=[],
    max_round=10,
)

In [10]:
manager = autogen.GroupChatManager(
    groupchat=groupchat, llm_config=llm_config
)


## Start the group chat!

<p style="background-color:#ECECEC; padding:15px; "> <b>Note:</b> In this lesson, you will use GPT 4 for better results. Please note that the lesson has a quota limit. If you want to explore the code in this lesson further, we recommend trying it locally with your own API key.

In [11]:
groupchat_result = user_proxy.initiate_chat(
    manager,
    message=task,
)

管理员 (to chat_manager):

撰写一篇关于英伟达在过去一个月股票价格表现的博客文章。今天的日期是2024-08-02。

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
规划者 (to chat_manager):

为了撰写一篇关于英伟达在过去一个月股票价格表现的博客文章，你可以按照以下步骤进行：

### 步骤 1: 获取英伟达股票的历史数据
使用Python代码从财经数据API（如Alpha Vantage、Yahoo Finance、或Alpha Advantage）获取英伟达（NVDA）在过去一个月的股票价格数据。

```python
import requests

api_key = 'YOUR_API_KEY'
symbol = 'NVDA'
function = 'TIME_SERIES_DAILY'
outputsize = 'compact'

url = f'https://www.alphavantage.co/query?function={function}&symbol={symbol}&outputsize={outputsize}&apikey={api_key}'
response = requests.get(url)
data = response.json()

# 提取过去一个月的数据
time_series = data['Time Series (Daily)']
```

### 步骤 2: 处理数据
将获取的数据转换为Pandas DataFrame，以便更容易进行分析和可视化。

```python
import pandas as pd

# 将数据转换为DataFrame
df = pd.DataFrame(time_series).T
df.index = pd.to_datetime(df.index)
df = df.astype(float)
df = df.sort_index()

# 获取过去一个月的数据
start_date = '2024-07-02'
end_date = '2024-08-02'

## Add a speaker selection policy

In [12]:
user_proxy = autogen.ConversableAgent(
    name="管理员",
    system_message="请给出任务，并向写作者发送"
    " 指令以完善博客文章。",
    code_execution_config=False,
    llm_config=llm_config,
    human_input_mode="ALWAYS",
)

planner = autogen.ConversableAgent(
    name="规划者",
    system_message="给定一个任务，请确定"
    " 完成该任务所需的信息。"
    " 请注意，所有信息都将通过"
    " Python 代码检索。请仅建议可以通过"
    " Python 代码检索的信息。"
    " 在其他人完成每个步骤后，检查进度并"
    " 指导剩余步骤。如果某个步骤失败，请尝试"
    " 解决方法。",
    description="给定一个任务，确定"
    " 完成该任务所需的信息。"
    " 在其他人完成每个步骤后，检查进度并"
    " 指导剩余步骤。",
    llm_config=llm_config,
)

engineer = autogen.AssistantAgent(
    name="工程师",
    llm_config=llm_config,
    description="根据规划者提供的计划"
    " 编写代码。",
)

writer = autogen.ConversableAgent(
    name="写作者",
    llm_config=llm_config,
    system_message="写作者。"
    " 请以 markdown 格式撰写博客（包含相关标题）"
    " 并将内容放入伪 ```md``` 代码块中。"
    " 您将接受管理员的反馈并完善您的博客。",
    description="在所有信息可用后，"
    " 根据代码执行结果撰写博客，并接受"
    " 管理员的反馈以完善博客。",
)

executor = autogen.ConversableAgent(
    name="执行者",
    description="执行工程师编写的代码并报告结果。",
    human_input_mode="NEVER",
    code_execution_config={
        "last_n_messages": 3,
        "work_dir": "coding",
        "use_docker": False,
    },
)

In [13]:
groupchat = autogen.GroupChat(
    agents=[user_proxy, engineer, writer, executor, planner],
    messages=[],
    max_round=10,
    allowed_or_disallowed_speaker_transitions={
        user_proxy: [engineer, writer, executor, planner],
        engineer: [user_proxy, executor],
        writer: [user_proxy, planner],
        executor: [user_proxy, engineer, planner],
        planner: [user_proxy, engineer, writer],
    },
    speaker_transitions_type="allowed",
)

In [14]:
manager = autogen.GroupChatManager(
    groupchat=groupchat, llm_config=llm_config
)

groupchat_result = user_proxy.initiate_chat(
    manager,
    message=task,
)

管理员 (to chat_manager):

撰写一篇关于英伟达在过去一个月股票价格表现的博客文章。今天的日期是2024-08-02。

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
规划者 (to chat_manager):

要撰写一篇关于英伟达在过去一个月股票价格表现的博客文章，你可以按照以下步骤进行：

### 步骤 1: 获取英伟达过去一个月的股票价格数据
使用Python代码从财经数据API（如Alpha Vantage, Yahoo Finance, 或 Quandl）获取英伟达（NVDA）在过去一个月的股票价格数据。以下是一个使用`yfinance`库的示例代码：

```python
import yfinance as yf
import pandas as pd

# 获取英伟达的股票数据
nvda = yf.Ticker("NVDA")

# 获取过去一个月的历史数据
hist = nvda.history(period="1mo")

# 打印数据
print(hist)
```

### 步骤 2: 分析数据
分析获取到的数据，计算关键指标，如开盘价、收盘价、最高价、最低价、交易量等。

```python
# 计算一些基本统计数据
summary = hist.describe()

# 打印摘要
print(summary)
```

### 步骤 3: 撰写博客文章
根据分析结果撰写博客文章。以下是一个简单的博客文章草稿：

---

**标题：英伟达（NVDA）过去一个月的股票价格表现分析**

**日期：2024-08-02**

在过去的一个月里，英伟达（NVDA）的股票价格经历了显著的波动。根据我们的数据分析，以下是一些关键的观察结果：

1. **开盘价与收盘价**：在过去一个月中，英伟达的开盘价平均为$X，收盘价平均为$Y。这表明股票在交易日结束时通常有$Z的变动。

2. **最高价与最低价**：最高价达到了$A，而最低价为$B。这显示了股票价格的波动范围。

3. **交易量**：平均交易量为$V，最高交易量达到了$W。交易量的增加通常与价

**Note**: You might experience slightly different interactions between the agents. The engineer agent may write incorrect code, which the executor agent will report and send back for correction. This process could go through multiple rounds.